# REI505M Machine Learning - Final project
### Due: --------

**Names**: Axel Kristján Axelsson, Bjarni Haukur Bjarnason <br />
**Email**: aka30@hi.is, bhb23@hi.is

**1. Image classification**

s


In [11]:
import numpy as np
from sklearn.model_selection import train_test_split
from PIL import Image
import tensorflow as tf

from data_loader import MyDataLoader
from neural_nets import NeuralNets
from helper_funcs import *


%load_ext autoreload 
%autoreload 2

norm = (128,128)

dl = MyDataLoader("combine2", norm)
nn = NeuralNets(norm)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
dl.normalize_train_data(norm_size=norm)

try:
    del X
    del y

except:
    pass

X, y = dl.get_lab_data()
dl.numpy_dump(X, "X")
dl.numpy_dump(y, "y")

In [2]:
# X = np.squeeze(dl.numpy_load("X"))
X = dl.numpy_load("X") # 0-1
y = dl.numpy_load("y") # -1 - 1
X_lab = X
y_lab = map_to(y)

In [ ]:
print(np.max(y))
print(np.min(y))
print(np.max(y_lab))
print(np.min(y_lab))
test = map_from(y_lab, y)
print(np.max(test))
print(np.min(test))

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X_lab, y_lab, test_size = 0.3, random_state = 3)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size = 0.33, random_state = 3)

In [ ]:
# from random import randint
from tensorflow.keras.optimizers import *
from tensorflow.keras.losses import *

# ?GANS svona
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.8)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

log_dir = "logs/fit/" + "model_e"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

#model = load_model("model_e")
model = nn.autoencoder()
model.compile(optimizer=RMSprop(), loss=MeanSquaredError())


percent = 0.6
size = X_train.shape[0]
for i in range(10):
    rand_array = np.random.choice(size, int(percent*size))
    X_bla = X_train[rand_array]
    y_bla = y_train[rand_array]
    model.fit(X_bla, y_bla, batch_size = 64, epochs = 5, validation_data=(X_val, y_val), callbacks=[tensorboard_callback])#, callbacks=[callback])
#save_model(model, "model_e", brave=True)



In [ ]:
%tensorboard --logdir logs/fit/

In [8]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import *
from keras import backend as K

log_dir = "logs/fit/" + "model_e_2"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


# data_gen_args = dict(rotation_range=30, fill_mode='nearest',
#                      width_shift_range=0.2, height_shift_range=0.2,
#                      horizontal_flip=True, vertical_flip=True,
#                      shear_range=0.3
# )
# image_datagen = ImageDataGenerator(**data_gen_args)
# mask_datagen = ImageDataGenerator(**data_gen_args)

# seed = 1337
# image_datagen.fit(X_train, augment=True, seed=seed)
# mask_datagen.fit(y_train, augment=True, seed=seed)

# image_generator = image_datagen.flow(X_train, seed = seed, batch_size=64)
# mask_generator = image_datagen.flow(y_train, seed = seed, batch_size=64)

# train_generator = zip(image_generator, mask_generator)


# # model = nn.autoencoder()
# # model.compile(optimizer=RMSprop(), loss=MeanSquaredError(), metrics=["accuracy"])
# # model = nn.model_s()
# # model.compile(optimizer='rmsprop', loss='mse', metrics=["accuracy"])

num_epochs = 30
img_number = 1511
test_image = np.expand_dims(X_test[img_number], 0)
minmax = np.zeros(shape=(1,1,2,2))
minmax[0,0,0,0] = np.max(y[:,:,:,0])
minmax[0,0,1,0] = np.min(y[:,:,:,0])
minmax[0,0,0,1] = np.max(y[:,:,:,1])
minmax[0,0,1,1] = np.min(y[:,:,:,1])

# input_shape = (128, 128, 1)
# batch_size = 32
# kernel_size = 3
# latent_dim = 256
# layer_filters = [64, 128, 256]
# layer_filters = layer_filters[::-1]

# inputs = Input(shape = input_shape)
# x = inputs
# for filters in layer_filters:
#     x = Conv2D(filters = filters,
#         kernel_size = kernel_size,
#         strides = 2,
#         activation ='relu',
#         padding ='same')(x)
  
# shape = K.int_shape(x)
# x = Flatten()(x)
# latent = Dense(latent_dim, name ='latent_vector')(x)
# encoder = Model(inputs, latent, name ='encoder')

# latent_inputs = Input(shape =(latent_dim, ), name ='decoder_input')
# x = Dense(shape[1]*shape[2]*shape[3])(latent_inputs)
# x = Reshape((shape[1], shape[2], shape[3]))(x)
# # stack of Conv2DTranspose(256)-Conv2DTranspose(128)-
# # Conv2DTranspose(64)
# for filters in layer_filters[::-1]:
#     x = Conv2DTranspose(filters = filters,
#                         kernel_size = kernel_size,
#                         strides = 2,
#                         activation ='relu',
#                         padding ='same')(x)
#     outputs = Conv2DTranspose(filters = 2,
#                             kernel_size = kernel_size,
#                             activation ='sigmoid',
#                             padding ='same',
#                             name ='decoder_output')(x)
#     decoder = Model(latent_inputs, outputs, name ='decoder')

#     autoencoder = Model(inputs, decoder(encoder(inputs)),
#                     name ='autoencoder')

model = nn.model_b()
model.compile(optimizer='rmsprop', loss='mse', metrics=["accuracy"])

percent = 0.4
size = X_train.shape[0]
for i in range(num_epochs):
    rand_array = np.random.choice(size, int(percent*size))
    X_bla = X_train[rand_array]
    y_bla = y_train[rand_array]
    #model.fit(x=X_train, y=y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val), callbacks=[tensorboard_callback])
    model.fit(x=X_bla, y=y_bla, epochs=10, batch_size=32, validation_data=(X_val, y_val), callbacks=[tensorboard_callback])
    pred = model.predict(test_image)
    save_images(test_image, map_from(pred, minmax), name="autoencode", enumerate=i)


# model.fit(x=X_train, y=y_train, epochs=1000, batch_size=32, validation_data=(X_val, y_val), callbacks=[tensorboard_callback])


C:\Users\bjarn\AppData\Roaming\Python\Python39\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 1/10


ValueError: in user code:

    C:\Users\bjarn\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py:853 train_function  *
        return step_function(self, iterator)
    C:\Users\bjarn\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\bjarn\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\bjarn\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\bjarn\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\bjarn\AppData\Roaming\Python\Python39\sit

In [4]:
model = load_model("model_e_3")
minmax = np.zeros(shape=(1,1,2,2))
minmax[0,0,0,0] = np.max(y[:,:,:,0])
minmax[0,0,1,0] = np.min(y[:,:,:,0])
minmax[0,0,0,1] = np.max(y[:,:,:,1])
minmax[0,0,1,1] = np.min(y[:,:,:,1])

In [6]:



rand_array = np.random.choice(X_test.shape[0], 3)
pred = model.predict(X_test[rand_array])
none_arr = np.zeros(shape=(3,128,128,2))
show_images(X_test[rand_array], none_arr)
show_images(X_test[rand_array], map_from(pred, minmax), enhance=True)

C:\Users\bjarn\AppData\Roaming\Python\Python39\site-packages\skimage\color\colorconv.py:1109: UserWarning: Color data out of range: Z < 0 in 2 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))


In [ ]:
save_model(model, "model_e_3", brave=True)


In [ ]:
# modell = load_model("model_e")
# none_arr = np.zeros(shape=(3, 128, 128, 2))
# modell = load_model("model_e_2")
rand_array = np.random.choice(X_test.shape[0], 3)
pred = model.predict(X_test[rand_array])
show_images(X_test[rand_array], map_from(y_test[rand_array], minmax))
show_images(X_test[rand_array], map_from(pred, minmax), enhance=True)
# save_images(X_test[rand_array], map_from(pred, y), name="pred", enhance=True)
# save_images(X_test[rand_array], map_from(y_test[rand_array], y), name="true")

In [ ]:
print(pred.shape)
print(np.max(pred))
print(np.min(pred))
new = map_from(pred, y_train)
print(np.max(new))
print(np.min(new))
# show_images(X_test[rand_array], map_from(pred, y_train))

In [ ]:
print(X_lab.shape)
print(y_lab.shape)
print(np.max(X_lab))
print(np.max(y_lab))
print(np.min(X_lab))
print(np.min(y_lab))


In [ ]:
datagen = ImageDataGenerator(rotation_range=30, fill_mode='nearest',
                             width_shift_range=0.2, height_shift_range=0.2,
                             horizontal_flip=True, vertical_flip=True,
                             brightness_range=[0.4,1.5],
                             
)

In [ ]:
from helper_funcs import *
arr = y_train[:10]
print(type(arr[0,0,0,0]))
print(np.max(arr))
print(np.min(arr))

test = map_to(arr)
print(type(test[0,0,0,0]))
print(np.max(test))
print(np.min(test))

In [ ]:
baka = map_from(test, arr)
print(type(baka[0,0,0,0]))
print(np.max(baka))
print(np.min(baka))

In [ ]:

images = np.zeros(shape=(10, 128, 128, 3))
images[:,:,:,0] = np.squeeze(X_train[:10], -1)
images[:,:,:,1:] = y_train[:10]
datagen = ImageDataGenerator(rotation_range=30, fill_mode='nearest',
                             width_shift_range=0.2, height_shift_range=0.2,
                             horizontal_flip=True, vertical_flip=True
)
image_it = datagen.flow(images, batch_size=10, shuffle=True)

X_next = image_it.next()[:,:,:,0]
y_next = image_it.next()[:,:,:,1:]
X_bla = np.expand_dims(np.array(X_next), -1)
y_bla = np.array(y_next)
print(X_bla.shape)
print(y_bla.shape)
print(np.max(y_bla))
print(np.min(y_bla))
y_bla = map_from(y_bla, y)
print(np.max(y_bla))
print(np.min(y_bla))
show_images(X_bla, y_bla)